# CDSADS_LSTM
바뀐 부분<br>
criterion = nn.CrossEntropyLoss()<br>
n_layers = 2<br>
self.i2h = nn.RNN(input_size + hidden_size, hidden_size, n_layers, dropout=0.3)<br>
model = RNN(n_class, n_hidden, n_categories)  # 57, 128, 18 문자수, 히든 수, 분류수<br>
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)<br>
optimizer.zero_grad()<br>
optimizer.step()<br>
사전학습모델: https://github.com/monologg/KoCharELECTRA<br>
https://www.marktechpost.com/2019/12/18/<br>classifying-the-name-nationality-of-a-person-using-lstm-and-pytorch/<br>

In [ ]:
##Parameter setting
setEpoch = 10
setLearningRate = 0.0001
setEpsilon = 1e-8
setBatch = 32
setMaxLength = 256
setSeed = 42
labelNumber = 2
setTry = 31

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
    
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
# Mount Google Drive to this Notebook instance.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for currentTry in range(30,setTry):
  import pandas as pd
  import os

  fileDir = "drive/My Drive/NLP/GH/AgentThemeFirst/Data/Binary/trainBi.csv"
  fr = open(fileDir, 'r')
  contents = fr.readlines()
  fr.close()

  train = pd.DataFrame(columns=('Label', 'Sentence'))
  i = 0
  label = ""
  sentence = ""
  for content in contents:
      if i == 0:
          pass
      else:
          infos = content.split(",")
          label = int(infos[2])
          sentence = infos[3].replace("\n", "")
          train.loc[i] = [label, sentence]
      i = i + 1


  train['Sentence'] = train['Sentence'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ",regex=True)
  train['Sentence'] = train['Sentence'].str.replace(r'\t+', " ", regex=True)
  train['Sentence'] = train['Sentence'].str.replace(r'[\\n]+', " ", regex=True)

  # 리뷰 문장 추출
  sentences = train['Sentence']

  # # 사전학습모델을 사용 안한 경우
  # # # word_list = list(set("".join(sentences).split()))
  # word_list = list(set("".join(list(sentences))))
  # word_dict = {w: i for i, w in enumerate(word_list)}
  # number_dict = {i: w for i, w in enumerate(word_list)}
  # n_class = len(word_dict)

  # CHILDES데이터 사용하여 사전학습 모델 튜닝
  entireCorpus = "drive/My Drive/NLP/GH/AgentThemeFirst/Data/Binary/CHILDES/CHILDES_all_spellchecked_corrected.txt"
  entireFr = open(entireCorpus, 'r')
  entireContents = entireFr.readlines()
  entireFr.close()

  import re

  entireSet = set()

  #음절을 추가
  for entireContent in entireContents:
      entireContent = re.sub('[^가-힣]', ' ', entireContent)
      entireContent = re.sub('[\s]+', '', entireContent)
      for each in entireContent:
          entireSet.add(each)

  # print("child:",len(entireSet))

  # 사전학습 모델 사용
  # 사전학습모델: https://github.com/monologg/KoCharELECTRA
  preFileDir = "drive/My Drive/NLP/GH/AgentThemeFirst/Data/Binary/Pretrained_model/vocab.txt"
  preFr = open(preFileDir, 'r')
  preContents = preFr.readlines()
  preFr.close()

  entireSet2 = set()

  for content in preContents:
    content = content.replace("\n","")
    if content != " ":
        entireSet.add(content)
        entireSet2.add(content)

  print("pre-trained:", len(entireSet2))
  print("child+pre-trained:",len(entireSet))

  word_list = []
  for content in entireSet:
      word_list.append(content.replace("\n", ""))
  word_dict = {w: i for i, w in enumerate(word_list)}
  number_dict = {i: w for i, w in enumerate(word_list)}
  n_class = len(word_dict)

  print(word_list)
  print(word_dict)
  print(number_dict)
  print(n_class)

  category_lines = {}
  all_categories = []
  for i in range(0,2):
      category = train['Label'] == i
      categoryName = "pattern"+str(i)
      category_lines[categoryName] = list(train[category]['Sentence'])
      all_categories.append(categoryName)

  n_categories = len(all_categories)
  print(category_lines)
  print(all_categories)





  import torch

  def charToIndex(char):
      return ''.join(word_list).find(char)

  def charToTensor(char):
      tensor = torch.zeros(1, n_class)
      tensor[0][charToIndex(char)] = 1
      return tensor

  def lineToTensor(line):
      tensor = torch.zeros(len(line), 1, n_class)
      for li, char in enumerate(line):
          tensor[li][0][charToIndex(char)] = 1
      return tensor

  import torch.nn as nn
  import torch
  torch.set_default_tensor_type('torch.cuda.FloatTensor')

  #os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
  #os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

  # 인풋-히든-아웃풋 형태의 신경망

  class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm_cell = nn.LSTM(input_size, hidden_size)  # LSTM cell
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, input, hidden):
        out, hidden = self.lstm_cell(input.view(1, 1, -1), hidden)
        output = self.h2o(hidden[0])
        output = self.softmax(output)
        return output.view(1, -1), hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size))

  n_hidden = 256 #RNN층의 히든 갯수   36, 64,  128
  model = LSTM(n_class, n_hidden, n_categories).to(device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')) # 57, 256, 2 음절 갯수, 히든 수, 분류수
  

  print(model)

  def categoryFromOutput(output):
      top_n, top_i = output.topk(1)  # 텐서의 가장 큰 값 및 주소
      category_i = top_i[0].item()  # 텐서에서 정수 값으로 변경
      return all_categories[category_i], category_i


  def trainingExample(cateNum, lineNum):
      category = all_categories[cateNum] # 0부터 24
      line = category_lines[category][lineNum]
      category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
      line_tensor = lineToTensor(line)
      return category, line, category_tensor, line_tensor

  ###############################모델학습 설정########################################
  # criterion = nn.NLLLoss()
  criterion = nn.CrossEntropyLoss() # change
  # learning_rate = 0.002  # 이것을 너무 높게 설정하면 발산할 수 있고, 너무 낮으면 학습이 되지 않을 수 있습니다.
  learning_rate = setLearningRate   # 0.0001 , 0.005, 0.001
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  def train(category_tensor, line_tensor):
      hidden = model.initHidden()

      optimizer.zero_grad()
      for i in range(line_tensor.size()[0]):
          output, hidden = model(line_tensor[i], hidden)

      loss = criterion(output, category_tensor)
      loss.backward()
      optimizer.step()

      # 매개변수의 경사도에 학습률을 곱해서 그 매개변수의 값에 더합니다.
      # for p in rnn.parameters():
      #     p.data.add_(p.grad.data, alpha=-learning_rate)

      return output, loss.item(), hidden


  ##############################결과구문으로출력###################################
  def outreault(guess):
      guess = int(guess)
      outConstruction = ""
      if guess == 0:
          outConstruction = "agent-first"
      elif guess == 1:
          outConstruction = "theme-first"

      return outConstruction


  ###############################모델학습########################################
  import time
  import math

  n_iters = setEpoch
  print_every = 10000
  plot_every = 500

  # 도식화를 위한 손실 추적
  current_loss = 0
  all_losses = []

  def timeSince(since):
      now = time.time()
      s = now - since
      m = math.floor(s / 60)
      s -= m * 60
      return '%dm %ds' % (m, s)

  start = time.time()

  eachClassLength = []
  for cateNum in range(0, len(all_categories)):
      currentCategory = all_categories[cateNum]
      eachClassLength.append(len(category_lines[currentCategory]))

  maxLength = max(eachClassLength) #-> 9패턴 1938

  outDir = "drive/My Drive/NLP/GH/AgentThemeFirst/Data/Binary/OutputLSTM/lstmBi_All_T"+ str(currentTry) +".csv"
  f = open(outDir, 'w')

  f.write("epoch,sentence,originalLabel,predictedLabel,predictedConstruction,result"+"\n")


  learningNum = 0
  for iter in range(1, n_iters + 1):

      maxLengthList = []
      for i in range(0, maxLength):
          maxLengthList.append(i)

      import random
      for i in range(0, maxLength):
          index = random.randint(0, len(maxLengthList) - 1)
          selectedID = maxLengthList[index]

          categoryNumList = []
          for cateNum in range(0, len(all_categories)):
              categoryNumList.append(cateNum)

          for j in range(0, n_categories):
              id = random.randint(0, len(categoryNumList) - 1)
              selectedIndex = categoryNumList[id]

              currentCategory = all_categories[selectedIndex]

              try:
                  category, line, category_tensor, line_tensor = trainingExample(selectedIndex, selectedID)
                  # print(category, line, category_tensor, line_tensor, learningNum)
                  output, loss, hidden = train(category_tensor, line_tensor)  # 각 학습단계의 정보가 담긴 hidden
                  current_loss += loss

                  learningNum = learningNum + 1

                  # iter 숫자, 손실, 이름, 추측 화면 출력
                  if learningNum % print_every == 0:
                      guess, guess_i = categoryFromOutput(output)
                      correct = '✓' if guess == category else '✗ (%s)' % category
                      print('%d %d%% (%s) %.4f %s / %s %s' % (
                          iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

                  # 현재 평균 손실을 전체 손실 리스트에 추가
                  if learningNum % plot_every == 0:
                      all_losses.append(current_loss / plot_every)
                      current_loss = 0
              except IndexError:
                  pass

              categoryNumList.remove(selectedIndex)

          maxLengthList.remove(selectedID)

      if iter == 1 or iter == 2 or iter == 3 or iter == 4 or iter == 5 or iter == 6 or iter == 7 or iter == 8 or iter == 9 or iter == 10:# or iter == 20 or iter == 30:# or iter == 40 or iter == 50:
          # ###############################전체 모델 성능 평가########################################

          print("-----" + str(iter) + " learning result-----")

          def evaluate(line_tensor):
              hidden = model.initHidden()

              for i in range(line_tensor.size()[0]):
                  output, hidden = model(line_tensor[i], hidden)

              return output

          def predict(input_line):
              # print('\n> %s' % input_line)
              with torch.no_grad():
                  output = evaluate(lineToTensor(input_line))
                  guess, guess_i = categoryFromOutput(output)

              return input_line, guess

          import pandas as pd

          testFileDir = "drive/My Drive/NLP/GH/AgentThemeFirst/Data/Binary/testBi_All.csv"
          testFr = open(testFileDir, 'r')
          testContents = testFr.readlines()
          testFr.close()

          test = pd.DataFrame(columns=('Label1', 'Label2', 'Sentence'))
          i = 0
          for content in testContents:
              if i == 0:
                  pass
              else:
                  infos = content.split(",")
                  label1 = int(infos[0])
                  label2 = int(infos[1])
                  sentence = infos[2].replace("\n", "")
                  test.loc[i] = [label1, label2, sentence]
              i = i + 1

          test['Sentence'] = test['Sentence'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+',
                                                          " ", regex=True)
          test['Sentence'] = test['Sentence'].str.replace(r'\t+', " ", regex=True)
          test['Sentence'] = test['Sentence'].str.replace(r'[\\n]+', " ", regex=True)

          # 리뷰 문장 추출
          testSentences = test['Sentence']

          totalNum = 0
          correctNum = 0
          for each in range(0, len(testSentences)):
              print(test['Label1'][each + 1], test['Label2'][each + 1], test['Sentence'][each + 1])
              sentence, guess = predict(test['Sentence'][each + 1])
              guess = guess.replace("pattern", "")
              if guess == str(test['Label1'][each + 1]) or guess == str(test['Label2'][each + 1]):
                  print("input: ", sentence, ", predict: ", guess, "(O)")
                  f.write(str(iter) + "," + sentence + "," + str(test['Label1'][each + 1]) + "or" + str(test['Label2'][each + 1]) + "," + guess + "," + outreault(guess)+ ",1" + "\n")
                  correctNum = correctNum + 1
              else:
                  f.write(str(iter) + "," + sentence + "," + str(test['Label1'][each + 1]) + "or" + str(test['Label2'][each + 1]) + "," + guess + "," + outreault(guess) + ",0" + "\n")
                  print("input: ", sentence, ", predict: ", guess, "(X)")
              totalNum = totalNum + 1

          print("totalNum: ", totalNum, " correctNum: ", correctNum, " accuracy: ", (correctNum / totalNum))

  f.close()


child: 1490
pre-trained: 11360
child+pre-trained: 11360
